# Formula Exploration Notebook V2

This notebook holds the attempt to explore formulas in Sympy further, especially custom ones.

## Basic custom formula

In [2]:
from sympy.abc import x
from sympy.utilities.lambdify import lambdify, implemented_function
from sympy.parsing.sympy_parser import parse_expr

custom_func = implemented_function('custom_func', lambda x: x+1)
lambdified_custom_func = lambdify(x, custom_func(x))

functions_dict = {
    'custom_func': lambdified_custom_func
}

# Test parsing
result = parse_expr('custom_func(4)', local_dict=functions_dict)
print(f'Result of custom function: {result}')

# Yes, this is working!

Result of custom function: 5


In [3]:
# What about nested functions?

nested_result = parse_expr('custom_func(custom_func(4))', local_dict=functions_dict)
print(f'Result of nested function: {nested_result}')

# Yeah, this works!

Result of nested function: 6


In [7]:
# What about parsing a function with variable?


variable_result = parse_expr('custom_func(a_1)', local_dict=functions_dict)
print(f'Result of custom function with variable: {variable_result}')


# Can we pass values to the variable?

new_dict = {'a_1': 10}
new_dict.update(functions_dict)


replaced_variable_result = parse_expr('custom_func(a_1)', local_dict=new_dict)
print(f'Result of custom function with filled variable: {replaced_variable_result}')

# Awesome, this works!

Result of custom function with variable: a_1 + 1
Result of custom function with filled variable: 11


## Variadic formulas

In Excel, there are variadic functions, for example: SUM(). How do we tackle this?

In [9]:
# What about the function above?


array_dict = {'a_1': [10, 20, 30]}
array_dict.update(functions_dict)


array_variable_result = parse_expr('custom_func(a_1)', local_dict=array_dict)
print(f'Result of custom function with filled variable: {array_variable_result}')

# Hmm. Based on this cell's error result, it seems the 'x' passed on the lambda could be anything.
# We can use type-checking and handle the operation accordingly.

TypeError: can only concatenate list (not "int") to list

In [20]:
# What about we make a function with array param?

array_func = implemented_function('array_func', lambda x: sum(x))
lambdified_array_func = lambdify(x, array_func(x))

test_array_dict = {
    'array_func': lambdified_array_func
}

array_func_result = parse_expr('array_func([1, 2, 3])', local_dict=test_array_dict)
print(f'Result of custom function with array param: {array_func_result}')

Result of custom function with array param: 6


In [24]:
# What about using lambdify right away, without implemented_function?

lambd_sum = lambdify(x, sum(x))

test_sum_dict = {
    'sum': lambd_sum
}

sum_result = parse_expr('sum([1, 2, 3])', local_dict=test_sum_dict)
print(f'Result of sum function parsing: {sum_result}')

# Ha! Of course this is failing, because Symbol is not iterable.
# So... unless we're using Sympy's predefined function, I think the
# implemented_function() method is necessary.

TypeError: 'Symbol' object is not iterable

# Comparing similar Excel formulas

Now that we can parse custom functions in Sympy, how should we tackle different yet similar formulas? Previously we have algebraic formula simplification, but we want to try if unit testing is the way to go here.

Unit testing here would be about assigning values to the variables in the string.


In [28]:
from sympy.abc import x
from sympy.utilities.lambdify import lambdify, implemented_function
from sympy.parsing.sympy_parser import parse_expr

sum_func = implemented_function('sum', lambda arr: sum(arr))
lambdified_sum_func = lambdify(x, sum_func(x))

functions_dict = {
    'sum': lambdified_sum_func,
    'A1': 5,
    'A2': 6,
    'A3': 7
}

# Test parsing
result = parse_expr('sum([A1, A2, A3])', local_dict=functions_dict)
print(f'Result of custom function: {result}')

# Yes, this is working!

Result of custom function: 18


In [1]:
# Importing from the implemented modules
from pysheetgrader.formula_parser import parse_formula

similar_formulas = [
    "=B2+B3+B4+B5",
    "=SUM(B2:B5)"
]

parsed_formulas = [parse_formula(f) for f in similar_formulas]

print("Parsed formulas:")
print(parsed_formulas)

Parsed formulas:
[b_2 + b_3 + b_4 + b_5, sum(b_3, b_2, b_5, b_4)]


In [2]:
# Now we have similar cell references, all we need to do is implementing sum, no?

from pysheetgrader.formula_parser import encode_cell_reference

cell_values = {
    "B2": 1,
    "B3": 2,
    "B4": 3,
    "B5": 4
}


encoded_cell_values = {encode_cell_reference(cr): cell_values[cr] for cr in cell_values}
print('Encoded cell values:')
print(encoded_cell_values)


Encoded cell values:
{'b_2': 1, 'b_3': 2, 'b_4': 3, 'b_5': 4}


In [42]:
# Implementing custom sum method properly

import sympy
from sympy.abc import x
from sympy.utilities.lambdify import lambdify, implemented_function
from sympy.parsing.sympy_parser import parse_expr

# Since we can't pass variadic parameters in lambdify, we might as well replace the occurence of sum with Add!
local_dict = {
    'sum': sympy.Add
}

local_dict.update(encoded_cell_values)

localized_parsed_formulas = [parse_formula(f, local_dict) for f in similar_formulas]

print("Localized parsed formulas:")
print(localized_parsed_formulas)

# If we want to force our way with lambdify, perhaps the parser needs to make all the parameters into an array. 
# That way, we could always access the variadic parameters by referencing the array element.

Localized parsed formulas:
[10, 10]


## Comparing unknown formulas with additions

What about unknown formulas with additions?

In [4]:
from sympy import simplify
from sympy.parsing.sympy_parser import parse_expr

formula_1 = parse_expr("SUM(B2, B5) + 1")
formula_2 = parse_expr("SUM(B2, B5) + 2 - 1")

is_equal_formulas = simplify(formula_1 - formula_2) == 0

print(f"Is the formula equal: {is_equal_formulas}")



Is the formula equal: True
